# IOP Example

In this notebook, the PYNQ Xlnk class will be used to allocate a memory buffer in the DDR memory. An IOP will be instantiated. An application will run on the IOP to modify the memory buffer in the DDR memory. 

Two files are provided in ``Session_4\iop`` *pynq_tutorial.bin* an IOP application, and *pynq_tutorial.py* a Python wrapper. Both these files should be copied to ``xilinx\pynq\iop`` on your board before running the example below.

In [ ]:
from time import sleep
from pprint import pprint

def get_kb(mmu):
    return int(mmu.cma_stats()['CMA Memory Available']//1024)

def get_bufcount(mmu):
    return int(memmanager.cma_stats()['Buffer Count'])

def print_kb(mmu):
    print("Available Memory (KB): " + str(get_kb(mmu)))

### Create an instance of Xlnk

An instance of Xlnk called *mmu* will be created. 

In [ ]:
from pynq import Xlnk
xlnk = Xlnk()

### Check the status of the mmu

*cma_stats()* can be used to get the status of the Xlnk instance. Xlnk can only allocate memory if sufficient space is available. 

In [ ]:
xlnk.cma_stats()

## Allocate memory buffer
Allocating the memory buffer returns the virtual address. The function cma_get_phy_addr() can be used to return the physical address. 

Allocate the memory, get the physical address, and print the status of the memory space before and after Xlnk allocates memory:

In [ ]:
import numpy as np 

print("Before memory allocation:")
print_kb(xlnk)

# Allocate memory
buffer = xlnk.cma_array(shape=(1000,), dtype=np.int32)

print("After memory allocation:")
print_kb(xlnk)

### Check the memory buffer addresses

The virtual address can be used by any application running in Linux. This could be a Python application, or a C/C++ or other application running in Linux. The Physical address can be passed to an IP block in an overlay.

## Instantiate the IOP

The IOP application waits until it receives a (physical) memory address, an initialization value, and a length. Once it receives these values, it will write the initialization value to the first location in the memory buffer, then increment the value and write it to the next N locations, where N is the length. 

Note that care should be taken with this example. The IOP has full access to DDR memory where the Linux OS is running. Giving the wrong buffer address could allow the IOP application to modify memory where the operating system is running which would cause the system to crash. 


**C code snippet of the IOP application:**

```c

   MAILBOX_DATA(BUFFER_ADDR)=0;

   while(1){
      while(MAILBOX_DATA(BUFFER_ADDR)==0); // Wait for buffer address
      
      // DDR is accessed through a GP port at offset 0x20000000
      buffer = (unsigned *)(MAILBOX_DATA(BUFFER_ADDR)|0x20000000); // Cast to pointer and convert to DDR offset address
      data = MAILBOX_DATA(INIT_DATA);
      length = MAILBOX_DATA(ARRAY_LENGTH);
      
      // Write memory buffer in DDR
      for(i=0; i<length; i++){
          buffer[i]= data+i;
      }
      
      MAILBOX_DATA(BUFFER_ADDR) = 0; // Reset buffer address
   }```

### Instantiate the IOP

Instantiate the IOP and start running the application. 

In [ ]:
from pynq import Overlay

overlay = Overlay("./bitstream/pynq_tutorial.bit")

In [ ]:
help(overlay)

In [ ]:
from iop.pynq_tutorial import Pynq_Tutorial
iop = Pynq_Tutorial(overlay.iop1.mb_info)

In [ ]:
help(iop)

Check the content of the buffer before sending the buffer address to the IOP.

In [ ]:
print("Initial state of the buffer")
length = 10
for i in range(length):
    print(buffer[i])    

## Write to buffer from IOP

Write the physical pointer address returned form the *mmu* Xlnk instance, along with an initialization value and a length. The IOP application will then write to the memory buffer.

In [ ]:
data = 1
length = 10
iop.write_to_buffer(buffer.physical_address, length, data)

Check the contents of the buffer after the IOP application has modified the buffer. The cell above can be re-run with different values of data and length.

In [ ]:
print("State of buffer after IOP write")
for i in range(length):
    print(buffer[i])

## Free Memory

The last step should be to free the memory. 

In [ ]:
from time import sleep
buffer.freebuffer()
sleep(1)
print(xlnk.cma_stats())
print_kb(xlnk)